In [5]:
from google.cloud import bigquery, storage
import fastavro
import io

In [9]:
project = "evident-data-dev"
chosen = "evident-data-dev-backups.lucas-test-exports"

try:
    bq_client = bigquery.Client(project=bq_project)
    datasets = bq_client.list_datasets()
    sets_tables_list = []
    
    for dataset in datasets:
        tables = bq_client.list_tables(dataset.dataset_id)
        sets_tables_list.append({"set_name":dataset.dataset_id, "tables_in_set":[table.table_id for table in tables]})

    schema = {
    "type": "record",
    "name": "BQAssets",
    "fields": [
        {"name": "set_name", "type": "string"},
        {"name": "tables_in_set", "type": {"type": "array", "items": "string"}},
    ],
    }
    bucket_name = "evident-data-dev-backups"
    file_path = "lucas-test-exports/bq_sets_and_tables.avro"
    
    bytes_buffer = io.BytesIO()
    fastavro.writer(bytes_buffer, schema, sets_tables_list)
    bytes_buffer.seek(0)

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(file_path)

    blob.upload_from_file(bytes_buffer, content_type='application/octet-stream')

    print("The avro file succesfully uploaded")
           
except Exception as e:
    print(f"An unexpected error occurred: {e}")

The avro file succesfully uploaded
